In [ ]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBCell5ParamV2, WBCell5ParamV3, WBCell4Param
import numpy as np
from numpy import sqrt, sin, cos, tan, arctan

# Demo showing kinematics of different waterbomb cells

## Waterbomb cells with 5 parameters (asymmetric cell)

### Cell V1

In [ ]:
wb = WBCell5Param()
wb.interact()

### Cell V2

In [ ]:
wb = WBCell5ParamV2()
wb.interact()

### Cell V3

In [ ]:
wb = WBCell5ParamV3(debug=True)
wb.interact()

In [ ]:
diff = wb.X_Ia[5]-wb.X_Ia[0]
print(diff)
np.sqrt(np.einsum('a, a', diff, diff))

In [ ]:
diff = wb.X_Ia[4]-wb.X_Ia[3]
print(diff)
np.sqrt(np.einsum('a, a', diff, diff))

In [ ]:
# Solve l2 analytically with sympy
# solutions of l2 where yro, ylo are negative are all the same, only 2 positive solutions are valid

import sympy as sp
a, b, c, d, m, l1, l2 = sp.symbols('a, b, c, d, m, l1, l2', real=True, nonnegative=True)
e = a - c
d = sp.sqrt(b ** 2 + e ** 2) # checked
k = sp.sqrt(c ** 2 - m ** 2) # checked
M = sp.Matrix([0, 0, -k]) # checked

xro = (l2 ** 2 - d ** 2) / (4 * m) # checked
zro = (a * c - k ** 2 - m * xro) / k # checked
yro = sp.sqrt(d ** 2 - zro ** 2 - (xro - m) ** 2) # checked

xlo = (d ** 2 - l1 ** 2) / (4 * m) # checked
zlo = (a * c - k ** 2 + m * xlo) / k # checked
ylo = sp.sqrt(d ** 2 - zlo ** 2 - (xlo + m) ** 2) # checked
# ylo = yro + sp.sqrt(4*a**2-(zro-zlo)**2-(xro-xlo)**2) # checked

G = xro * xlo + yro * ylo + (zro +k)* (zlo+k) + (a**2-b**2) # checked
sol = sp.solve(G, l2, simplify=False)
sol

In [ ]:
sol_simple = [sp.simplify(sol[i]) for i in range(len(sol))]
sol_simple

In [ ]:
sol[1].subs([(a, 500), (b, 500), (c, 500), (m, 250), (l1, 750)]).evalf()

In [ ]:
# Solve l2 numerically

from scipy.optimize import root
a = 500
b = 500
c = 500
m = 250
l1 = 750

e = a - c
d = np.sqrt(b ** 2 + e ** 2) # checked
k = np.sqrt(c ** 2 - m ** 2) # checked
M = np.array([0, 0, -k]) # checked

def get_G(l2, yro_positive=True, ylo_positive=True):
    print(l2, yro_positive, ylo_positive)
    xro = (l2 ** 2 - d ** 2) / (4 * m)  # checked
    zro = (a * c - k ** 2 - m * xro) / k  # checked

    yro_quad = d ** 2 - zro ** 2 - (xro - m) ** 2  # checked
    if yro_quad < 0:
        return np.nan
    yro = np.sqrt(yro_quad) if yro_positive else -np.sqrt(yro_quad) # using np.abs(yro_quad) like before would introduce new wrong solutions

    xlo = (d ** 2 - l1 ** 2) / (4 * m)  # checked
    zlo = (a * c - k ** 2 + m * xlo) / k  # checked

    ylo_quad = d ** 2 - zlo ** 2 - (xlo + m) ** 2  # checked
    if ylo_quad < 0:
        return np.nan
    ylo = np.sqrt(ylo_quad) if ylo_positive else -np.sqrt(ylo_quad) # using np.abs(yro_quad) like before would introduce new wrong solutions

    G = xro * xlo + yro * ylo + (zro + k) * (zlo + k) + (a ** 2 - b ** 2)  # checked

    return G

# l2 = root(lambda l2: get_coords(l2), 0, tol=1e-6).x # gives negative solution
l2_max = 2000
l2 = np.nan
for l2_i in np.linspace(0, l2_max, 100):
    sol = root(get_G, l2_i, tol=1e-6, args=(True, True))
    if sol.success and sol.x[0] >= 0:
        l2 = sol.x
        break
if l2 == np.nan:
    for l2_i in np.linspace(0, l2_max, 100):
        sol = root(get_G, l2_i, tol=1e-6, args=(True, False))
        if sol.success and sol.x[0] >= 0:
            l2 = sol.x
            break
if l2 == np.nan:
    for l2_i in np.linspace(0, l2_max, 100):
        sol = root(get_G, l2_i, tol=1e-6, args=(False, True))
        if sol.success and sol.x[0] >= 0:
            l2 = sol.x
            break
if l2 == np.nan:
    for l2_i in np.linspace(0, l2_max, 100):
        sol = root(get_G, l2_i, tol=1e-6, args=(False, False))
        if sol.success and sol.x[0] >= 0:
            l2 = sol.x
            break
l2
# right l2 sol should be l2 = 940.25

In [ ]:
a = 500
b = 500
c = 500
m = 0.5 # 0-1
l1 = 750

# Control folding state
m = m * c

# c = a -e
e = a - c
# b = sqrt(d**2 - e**2)

d = sqrt(b ** 2 + e ** 2)
print('d: ', d)

# k is a short cut
k = sqrt(c ** 2 - m ** 2)
M = np.array([0, 0, -k])  # Mittelpunkt

# Sol 1:
# l2_quad = (c**6+(-a**2-5*b**2-2*l1**2)*c**4+16*a*b**2*c**3+(-a**4+(-6*b**2-4*l1**2-8*m**2)*a**2-5*b**4+8*b**2*m**2+l1**4)*c**2-16*a*m**2*(a**2+b**2)*c+a**6+(3*b**2-2*l1**2-8*m**2)*a**4+(l1**4+(-4*b**2+8*m**2)*l1**2+3*b**4)*a**2+b**2*l1**4+(-2*b**4-8*b**2*m**2)*l1**2+b**6+8*b**4*m**2+2*((a**2+2*a*c+b**2+c**2-l1**2)*(a**4+2*a**3*c+(2*b**2+c**2-l1**2)*a**2-6*a*b**2*c+b**2*(b**2+c**2-l1**2))*(a**4*c**2+(-4*c**3+8*c*m**2)*a**3+(2*b**2*c**2+6*c**4+(-2*l1**2-16*m**2)*c**2+16*m**4)*a**2-4*c*(c**2-2*m**2)*(b**2+c**2-l1**2)*a+b**4*c**2+(2*c**4+(-2*l1**2-16*m**2)*c**2+16*m**4)*b**2+c**2*(c-l1)**2*(c+l1)**2))**(1/2))/(a**2+b**2-c**2+2*c*l1-l1**2)/(a**2+b**2-c**2-2*c*l1-l1**2)
# l2 = np.sqrt(l2_quad)

# Sol 2:
# l2_quad = (c**6+(-a**2-5*b**2-2*l1**2)*c**4+16*a*b**2*c**3+(-a**4+(-6*b**2-4*l1**2-8*m**2)*a**2-5*b**4+8*b**2*m**2+l1**4)*c**2-16*a*m**2*(a**2+b**2)*c+a**6+(3*b**2-2*l1**2-8*m**2)*a**4+(l1**4+(-4*b**2+8*m**2)*l1**2+3*b**4)*a**2+b**2*l1**4+(-2*b**4-8*b**2*m**2)*l1**2+b**6+8*b**4*m**2+2*((a**2+2*a*c+b**2+c**2-l1**2)*(a**4+2*a**3*c+(2*b**2+c**2-l1**2)*a**2-6*a*b**2*c+b**2*(b**2+c**2-l1**2))*(a**4*c**2+(-4*c**3+8*c*m**2)*a**3+(2*b**2*c**2+6*c**4+(-2*l1**2-16*m**2)*c**2+16*m**4)*a**2-4*c*(c**2-2*m**2)*(b**2+c**2-l1**2)*a+b**4*c**2+(2*c**4+(-2*l1**2-16*m**2)*c**2+16*m**4)*b**2+c**2*(c-l1)**2*(c+l1)**2))**(1/2))/(a**2+b**2-c**2+2*c*l1-l1**2)/(a**2+b**2-c**2-2*c*l1-l1**2)
# l2 = np.sqrt(l2_quad)

l2 = sqrt((a**6 + 3*a**4*b**2 - a**4*c**2 - 2*a**4*l1**2 - 8*a**4*m**2 - 16*a**3*c*m**2 + 3*a**2*b**4 - 6*a**2*b**2*c**2 - 4*a**2*b**2*l1**2 - a**2*c**4 - 4*a**2*c**2*l1**2 - 8*a**2*c**2*m**2 + a**2*l1**4 + 8*a**2*l1**2*m**2 + 16*a*b**2*c**3 - 16*a*b**2*c*m**2 + b**6 - 5*b**4*c**2 - 2*b**4*l1**2 + 8*b**4*m**2 - 5*b**2*c**4 + 8*b**2*c**2*m**2 + b**2*l1**4 - 8*b**2*l1**2*m**2 + c**6 - 2*c**4*l1**2 + c**2*l1**4 - 2*sqrt(a**10*c**2 + 8*a**9*c*m**2 + 5*a**8*b**2*c**2 - 4*a**8*c**4 - 4*a**8*c**2*l1**2 + 16*a**8*c**2*m**2 + 16*a**8*m**4 - 8*a**7*b**2*c**3 + 32*a**7*b**2*c*m**2 - 8*a**7*c**3*m**2 - 24*a**7*c*l1**2*m**2 + 64*a**7*c*m**4 + 10*a**6*b**4*c**2 + 8*a**6*b**2*c**4 - 16*a**6*b**2*c**2*l1**2 - 32*a**6*b**2*c**2*m**2 + 64*a**6*b**2*m**4 + 6*a**6*c**6 + 4*a**6*c**4*l1**2 - 32*a**6*c**4*m**2 + 6*a**6*c**2*l1**4 - 32*a**6*c**2*l1**2*m**2 + 96*a**6*c**2*m**4 - 32*a**6*l1**2*m**4 - 24*a**5*b**4*c**3 + 48*a**5*b**4*c*m**2 + 8*a**5*b**2*c**5 + 24*a**5*b**2*c**3*l1**2 - 56*a**5*b**2*c**3*m**2 - 72*a**5*b**2*c*l1**2*m**2 + 64*a**5*b**2*c*m**4 - 8*a**5*c**5*m**2 - 16*a**5*c**3*l1**2*m**2 + 64*a**5*c**3*m**4 + 24*a**5*c*l1**4*m**2 - 64*a**5*c*l1**2*m**4 + 10*a**4*b**6*c**2 + 32*a**4*b**4*c**4 - 24*a**4*b**4*c**2*l1**2 - 128*a**4*b**4*c**2*m**2 + 96*a**4*b**4*m**4 - 30*a**4*b**2*c**6 - 36*a**4*b**2*c**4*l1**2 + 32*a**4*b**2*c**4*m**2 + 18*a**4*b**2*c**2*l1**4 + 96*a**4*b**2*c**2*l1**2*m**2 - 32*a**4*b**2*c**2*m**4 - 96*a**4*b**2*l1**2*m**4 - 4*a**4*c**8 + 4*a**4*c**6*l1**2 + 16*a**4*c**6*m**2 + 4*a**4*c**4*l1**4 - 32*a**4*c**4*l1**2*m**2 + 16*a**4*c**4*m**4 - 4*a**4*c**2*l1**6 + 16*a**4*c**2*l1**4*m**2 - 32*a**4*c**2*l1**2*m**4 + 16*a**4*l1**4*m**4 - 24*a**3*b**6*c**3 + 32*a**3*b**6*c*m**2 - 16*a**3*b**4*c**5 + 48*a**3*b**4*c**3*l1**2 + 40*a**3*b**4*c**3*m**2 - 72*a**3*b**4*c*l1**2*m**2 - 64*a**3*b**4*c*m**4 + 8*a**3*b**2*c**7 + 16*a**3*b**2*c**5*l1**2 - 48*a**3*b**2*c**5*m**2 - 24*a**3*b**2*c**3*l1**4 + 48*a**3*b**2*c*l1**4*m**2 + 8*a**3*c**7*m**2 - 24*a**3*c**5*l1**2*m**2 + 24*a**3*c**3*l1**4*m**2 - 8*a**3*c*l1**6*m**2 + 5*a**2*b**8*c**2 + 24*a**2*b**6*c**4 - 16*a**2*b**6*c**2*l1**2 - 96*a**2*b**6*c**2*m**2 + 64*a**2*b**6*m**4 + 34*a**2*b**4*c**6 - 52*a**2*b**4*c**4*l1**2 + 32*a**2*b**4*c**4*m**2 + 18*a**2*b**4*c**2*l1**4 + 160*a**2*b**4*c**2*l1**2*m**2 - 96*a**2*b**4*c**2*m**4 - 96*a**2*b**4*l1**2*m**4 + 16*a**2*b**2*c**8 - 40*a**2*b**2*c**6*l1**2 - 64*a**2*b**2*c**6*m**2 + 32*a**2*b**2*c**4*l1**4 + 128*a**2*b**2*c**4*l1**2*m**2 + 32*a**2*b**2*c**4*m**4 - 8*a**2*b**2*c**2*l1**6 - 64*a**2*b**2*c**2*l1**4*m**2 - 64*a**2*b**2*c**2*l1**2*m**4 + 32*a**2*b**2*l1**4*m**4 + a**2*c**10 - 4*a**2*c**8*l1**2 + 6*a**2*c**6*l1**4 - 4*a**2*c**4*l1**6 + a**2*c**2*l1**8 - 8*a*b**8*c**3 + 8*a*b**8*c*m**2 - 24*a*b**6*c**5 + 24*a*b**6*c**3*l1**2 + 88*a*b**6*c**3*m**2 - 24*a*b**6*c*l1**2*m**2 - 64*a*b**6*c*m**4 - 24*a*b**4*c**7 + 48*a*b**4*c**5*l1**2 + 88*a*b**4*c**5*m**2 - 24*a*b**4*c**3*l1**4 - 112*a*b**4*c**3*l1**2*m**2 - 64*a*b**4*c**3*m**4 + 24*a*b**4*c*l1**4*m**2 + 64*a*b**4*c*l1**2*m**4 - 8*a*b**2*c**9 + 24*a*b**2*c**7*l1**2 + 8*a*b**2*c**7*m**2 - 24*a*b**2*c**5*l1**4 - 24*a*b**2*c**5*l1**2*m**2 + 8*a*b**2*c**3*l1**6 + 24*a*b**2*c**3*l1**4*m**2 - 8*a*b**2*c*l1**6*m**2 + b**10*c**2 + 4*b**8*c**4 - 4*b**8*c**2*l1**2 - 16*b**8*c**2*m**2 + 16*b**8*m**4 + 6*b**6*c**6 - 12*b**6*c**4*l1**2 - 32*b**6*c**4*m**2 + 6*b**6*c**2*l1**4 + 32*b**6*c**2*l1**2*m**2 + 32*b**6*c**2*m**4 - 32*b**6*l1**2*m**4 + 4*b**4*c**8 - 12*b**4*c**6*l1**2 - 16*b**4*c**6*m**2 + 12*b**4*c**4*l1**4 + 32*b**4*c**4*l1**2*m**2 + 16*b**4*c**4*m**4 - 4*b**4*c**2*l1**6 - 16*b**4*c**2*l1**4*m**2 - 32*b**4*c**2*l1**2*m**4 + 16*b**4*l1**4*m**4 + b**2*c**10 - 4*b**2*c**8*l1**2 + 6*b**2*c**6*l1**4 - 4*b**2*c**4*l1**6 + b**2*c**2*l1**8))/(a**4 + 2*a**2*b**2 - 2*a**2*c**2 - 2*a**2*l1**2 + b**4 - 2*b**2*c**2 - 2*b**2*l1**2 + c**4 - 2*c**2*l1**2 + l1**4))

# l2 = 940.236

if True:
    print('l2=', l2)

xro = (l2 ** 2 - d ** 2) / (4 * m)
zro = (a * c - c ** 2 + m ** 2 - m * xro) / k
yro = sqrt(np.abs(d ** 2 - zro ** 2 - (xro - m) ** 2))


xlo = -(l1 ** 2 - d ** 2) / (4 * m)
zlo = (a * c - c ** 2 + m ** 2 + m * xlo) / k
ylo = sqrt(np.abs(d ** 2 - zlo ** 2 - (xlo + m) ** 2))

print('xlo, ylo, zlo:', xlo, ylo, zlo)

# Vr und Vl liegen auf der x-Achse
Uru = np.array([-xlo, -ylo, zlo])
Ulu = np.array([-xro, -yro, zro])
Uro = np.array([xro, yro, zro])
Ulo = np.array([xlo, ylo, zlo])
Vr = np.array([m, 0, 0])
Vl = np.array([-m, 0, 0])

if False:
    print(str(round(np.abs(c - (a ** 2 + b ** 2) ** 0.5), 1)), '<= l1  <=', str(round(c + (a ** 2 + b ** 2) ** 0.5, 1)))

X_Ia = np.vstack((M, Uru, Ulu, Uro, Ulo, Vr, Vl)).astype(np.float32)
X_Ia

In [ ]:
diff = X_Ia[3]-X_Ia[4]

np.sqrt(np.einsum('a, a', diff, diff))

## Waterbomb cell with 4 parameters (symmetric cell)

In [ ]:
wb = WBCell4Param()
wb.interact()

# Cell v3 Derivation

In [ ]:
import k3d
import random
def add_cell(plot, X_Ia, I_Fi):
    wb_mesh = k3d.mesh(X_Ia.astype(np.float32),
                             I_Fi.astype(np.uint32),
                            opacity=0.8,
                             color=0x999999,
                             side='double')
    rand_color = random.randint(0, 0xFFFFFF)
    plot += wb_mesh

    # wb_points = k3d.points(X_Ia.astype(np.float32),
    #                          color=0x999999,
    #                        point_size=100)
    # plot +=wb_points

    for I, X_a in enumerate(X_Ia):
        k3d_text = k3d.text('%g' % I, tuple(X_a), label_box=False, size=0.8, color=rand_color)
        plot += k3d_text

    wb_mesh_wireframe = k3d.mesh(X_Ia.astype(np.float32),
                                    I_Fi.astype(np.uint32),
                                    color=0x000000,
                                    wireframe=True)
    plot += wb_mesh_wireframe

In [ ]:
# Control geometry
a = 200
b = 200
c = 200

# Control folding state
m = 150   # x of Vr, -x of Vl
l1 = 250

# c = a -e
e = a - c
# b = sqrt(d**2 - e**2)
d = sqrt(b**2 + e**2)
# k is a short cut
k = sqrt(c**2-m**2)
M = np.array([0,0,-k]) # Mittelpunkt

l2 = 300
# l2 = ((-8*a**4*l1**2+8*a**4*d**2+8*d**2*e**4-4*e**4*l1**2-2*d**4*l1**2+d**2*l1**4+d**6+16*a**3*e*l1**2-16*a**3*d**2*e+16*a**2*d**2*e**2-12*a**2*e**2*l1**2-16*a*d**2*e**3+8*a*e**3*l1**2-32*a**4*m**2+32*a**3*e*m**2+2*a**2*l1**4-2*a*e*l1**4-4*a**2*d**2*l1**2+4*d**2*e**2*l1**2+8*e**2*l1**2*m**2-8*d**2*l1**2*m**2+8*a**2*l1**2*m**2+2*((4*a**4-4*a**3*e-3*a**2*d**2+4*a**2*e**2-a**2*l1**2+4*a*d**2*e-4*a*e**3+d**4-d**2*e**2-d**2*l1**2+e**2*l1**2)*(4*a**2-4*a*e+d**2-l1**2)*(a**2*d**4-2*a**2*d**2*l1**2-8*a**2*d**2*m**2+16*a**2*e**2*m**2+a**2*l1**4-8*a**2*l1**2*m**2+16*a**2*m**4-2*a*d**4*e+4*a*d**2*e*l1**2+24*a*d**2*e*m**2-32*a*e**3*m**2-2*a*e*l1**4+8*a*e*l1**2*m**2+d**4*e**2-2*d**2*e**2*l1**2-16*d**2*e**2*m**2+16*d**2*m**4+16*e**4*m**2+e**2*l1**4-16*e**2*m**4))**(1/2)+10*a*d**4*e-8*a**2*d**2*m**2-8*d**2*e**2*m**2+8*d**4*m**2-2*a**2*d**4-8*d**4*e**2)/(2*a*e-2*a*l1+d**2-2*e**2+2*e*l1-l1**2)/(2*a*e+2*a*l1+d**2-2*e**2-2*e*l1-l1**2))**(1/2)
print(l2)


xro = (l2**2 -d**2)/(4 * m)
zro = (a*c - c**2 + m**2 - 1/4*(l2**2-d**2))/k
yro = sqrt( (c-a)**2 + b**2 - zro**2 - (xro-m)**2 )

xlo = -(l1**2-d**2) / (4*m)
zlo = (a*c - c**2 +m**2 +m*xlo)/k
ylo = sqrt( l1**2 - zlo**2 -(xlo-m)**2)

# Vr und Vl liegen auf der x-Achse
Uru = np.array([-xlo,-ylo,zlo])
Ulu = np.array([-xro,-yro,zro])
Uro = np.array([xro,yro,zro])
Ulo = np.array([xlo,ylo,zlo])
Vr = np.array([m,0,0])
Vl = np.array([-m,0,0])

I_Fi = np.array([[0, 1, 2], [0, 3, 4], [0, 1, 5], [0, 5, 3], [0, 2, 6], [0, 6, 4]]).astype(np.int32)
X_Ia = np.vstack((M, Uru, Ulu, Uro, Ulo, Vr, Vl)).astype(np.float32)
X_Ia

In [ ]:
plot = k3d.plot()
add_cell(plot, X_Ia, I_Fi)
plot